# I. Supervised Hidden Markov Model

### 1. Prepare dataset

import numpy as np
# raw_text = """Hươu là loài vật được con_người thuần_dưỡng đã hàng trăm năm .
# Nhưng cũng là loài vật nhút_nhát , hễ có tiếng_động là lập_tức tập_trung thính_lực vào đôi tai .
# Tai hươu có_thể quay 4 hướng .
# Hươu sẵn_sàng tự_vệ bằng cách bỏ chạy khi nghe có tiếng_động lạ ."""
#states: start word is 1, non start word is 2
def sentence_to_words_and_states(s):
  words = []
  states = []
  for raw_word in s.split():
    if not "_" in raw_word:
      words.append(raw_word)
#       if not raw_word in list(string.punctuation):       
#         states.append(1)
#       else:
#         states.append(3)
      states.append(1)
    else:
      syllables = raw_word.split("_")
      words = words + syllables
      states.append(2)
      for i in range(0, len(syllables) - 1):
        states.append(3)
  assert(len(words) == len(states))
  return words, states

def preprocess_sentence(raw_text):
  raw_sentences = raw_text.split(".")[:-1]
  sentences = [sentence for sentence in raw_sentences if len(sentence.strip()) > 0]
  lower_sentences = [s.lower() for s in sentences]
  return lower_sentences

def prepare_sentence_words_and_states(raw_text):
  sentences = preprocess_sentence(raw_text)
  sentence_words = []
  sentence_states = []
  for s in sentences:
    words, states = sentence_to_words_and_states(s)
    sentence_words.append(words)
    sentence_states.append(states)
  return sentences, sentence_words, sentence_states

def prepare_sentence_words(raw_text):
  sentences = preprocess_sentence(raw_text)
  for s in sentences:
    words, _ = sentence_to_words_and_states(s)
    sentence_words.append(words)
  return sentence_words

In [119]:
import pandas as pd
import os

def read_raw_text(dataset_path, output_folder):
  output_path = os.path.join(dataset_path, output_folder)
  file_paths = [os.path.join(output_path, file_name) for file_name in os.listdir(output_path)]
  raw_text = ""
  for file_path in file_paths:
    f = open(file_path, 'r')
    raw_text += f.read()
    f.close()
  return raw_text
raw_text = read_raw_text("datasets", "output") 
sentences, sentence_words, sentence_states = prepare_sentence_words_and_states(raw_text)
# sentence_words = prepare_sentence_words(raw_text)
# sentence_words[1]
# for i in range(10, 16):
#   print(sentences[i])
#   print(sentence_states[i])
#   print('\n')
#   assert(len(sentence_words[i]) == len(sentence_states[i]))
print(sentences[0])
print(sentence_states[0])

hươu là loài vật được con_người thuần_dưỡng đã hàng trăm năm 
[1, 1, 1, 1, 1, 2, 3, 2, 3, 1, 1, 1, 1]


In [120]:
n_words = 3500
n_states = 3
only_word_state = [1]
start_word_state = [2]
in_word_state = [3]

In [4]:
import collections

def build_dataset(sentences, n_words):
  words = [word for sentence in sentences for word in sentence]
  count = [["UNK", -1]]
  count.extend(collections.Counter(words).most_common(n_words - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = []
  unk_count = 0
  for sentence in sentences:
    index_sentence = []
    for word in sentence:
      index = dictionary.get(word, 0)
      if index == 0:
        unk_count += 1
      index_sentence.append(index)
    data.append(index_sentence)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return count, data, dictionary, reverse_dictionary
count, data, dictionary, reverse_dictionary = build_dataset(sentence_words, n_words)

data = np.asarray(data)
sentence_states = np.asarray(sentence_states)
sentences = np.asarray(sentences)
random_indices = np.random.permutation(len(data))
data = data[random_indices]
sentence_states = sentence_states[random_indices]
sentences = sentences[random_indices]
test_size = 500
X_train = data[:-test_size]
y_train = sentence_states[:-test_size]
X_val = data[-test_size:]
y_val = sentence_states[-test_size:]

In [157]:
import pickle
def save_obj(obj, name):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
save_obj(dictionary, 'dictionary')
save_obj(reverse_dictionary, 'reverse_dictionary')

### 2. Training

In [5]:
def build_probability_matrix(data, sentence_states):
  # A[b,a] represents P(y^j = b|y^(j-1) = a), with first row and first column represents the Start state
  # O[w,a] represents P(x^j = w|y^j = a), with first column represents the Start state
  A = np.zeros((n_states+1, n_states+1))
  O = np.zeros((n_words, n_states+1))
  for sentence_num in range(len(data)):
    for word_num in range(0, len(data[sentence_num])):
      previous_state = 0 if word_num == 0 else sentence_states[sentence_num][word_num - 1]
      A[sentence_states[sentence_num][word_num], previous_state] += 1
  A = A / np.sum(A, axis = 0, keepdims=True)

  for sentence_num in range(len(data)):
    for word_num in range(0, len(data[sentence_num])):
      O[data[sentence_num][word_num], sentence_states[sentence_num][word_num]] += 1
  O[:,1:] = O[:,1:] / np.sum(O[:,1:], axis = 0, keepdims=True)
  return A, O
A, O = build_probability_matrix(data, sentence_states)

### 3. Inference

In [7]:
# Vertibi algorithm for inference
def find_best_sequence_markov(A, O, sentence_encoded):
  M = len(sentence_encoded)
  # best_score[a][b] is the best log probability of sequence ending at position = b having state = a
  # best_state[a][b] is the state (from 0) of the position before b that gives max log probability at position b
  best_score = np.zeros((n_states, M))
  best_state = np.zeros((n_states, M)).astype(int)
  for current_word_index in range(M):
    with np.errstate(divide='ignore'):
      if current_word_index == 0:
        best_score[:, 0] = [np.log(A[state][0]) + np.log(O[sentence_encoded[0]][state])
                               for state in range(1, n_states + 1)]
      else:
        for current_state in range(1, n_states + 1):
          current_state_log_score = [np.log(A[current_state][previous_state]) + 
                                     np.log(O[sentence_encoded[current_word_index]][current_state])
                                     for previous_state in range(1, n_states + 1)]
          current_state_score = best_score[:, current_word_index - 1] + current_state_log_score
          best_score[current_state - 1, current_word_index] = np.max(current_state_score)
          best_state[current_state - 1, current_word_index] = np.argmax(current_state_score)
  # backtrack to find best_sequence
  best_sequence = []
  current_best_state = np.argmax(best_score[:, M - 1])
  best_sequence = [current_best_state + 1]
  #print(best_state)
  for current_word_index in range(M - 1, 0, -1):
    #print("current_best_state: ", current_best_state, "current_word_index: ", current_word_index)
    previous_best_state = best_state[current_best_state, current_word_index]
    best_sequence = [previous_best_state + 1] + best_sequence
    current_best_state = previous_best_state
  return best_sequence

In [146]:
import string
def print_word_in_sentence(sentence_encoded, sentence_state, reverse_dictionary):
  s = ""
  M = len(sentence_encoded)
  for word_index in range(M):
    current_word = reverse_dictionary[sentence_encoded[word_index]]
    if sentence_state[word_index] in only_word_state:
      if word_index < M - 1:
        s += current_word + " "
      else:
        s += current_word
    elif sentence_state[word_index] in start_word_state:
      if word_index < M - 1 and sentence_state[word_index + 1] in in_word_state:
        s += current_word + "_"
      elif word_index < M - 1:
        s += current_word + " "
      else:
        s += current_word
    else:
      if word_index < M - 1 and sentence_state[word_index + 1] in in_word_state:
        s += current_word + "_"
      elif word_index < M - 1:
        s += current_word + " "
      else:
        s += current_word
  return s

# II Conditional Random Field

### 1. Cost function

Create transition feature and matrix

In [122]:
import numpy as np
from scipy.sparse import csr_matrix
n_words, n_states = 3500, 3
# Input: yj_prev: previous state, yj: current state, xj: current word
# Output: vector phi representing the transition from previous state to current state given current word
# n_states does not include Start
def create_transition_feature(yj_prev, yj, xj):
  assert(yj_prev <= n_states and yj <= n_states and xj < n_words)
  phi_1 = np.zeros((n_states * n_words, 1))
  phi_2 = np.zeros(((n_states + 1) * n_states, 1))
  phi_1[(yj - 1) * n_words + xj, 0] = 1
  phi_2[(yj - 1) * (n_states + 1) + yj_prev, 0] = 1
  phi = np.concatenate((phi_1, phi_2), axis = 0)
  return phi

# def create_transition_feature_2(yj_prev, yj, xj):
#   assert(yj_prev <= n_states and yj <= n_states and xj < n_words)
#   row = np.array([(yj - 1) * n_words + xj, n_states * n_words + (yj - 1) * (n_states + 1) + yj_prev])
#   col = np.array([0, 0])
#   data = np.array([1, 1])
#   return csr_matrix((data, (row, col)), shape=(n_states * n_words + (n_states + 1) * n_states, 1))

def transition_position(yj_prev, yj, xj):
  return ((yj - 1) * n_words + xj, 0), (n_states * n_words + (yj - 1) * (n_states + 1) + yj_prev, 0)

Compute numerator F

In [123]:
# feature function a position j
def feature_function(w, theta, yj, yj_prev, xj, xj_prev):
  f = 0
  p1, p2 = transition_position(yj_prev, yj, xj)
  f += w[p1] + w[p2]
  if xj_prev != -1:
    sign = 1 if yj in in_word_state else -1
    f += sign * np.dot(theta[xj, :], theta[xj_prev, :])
  return f
    
def compute_F(w, theta, y, x):
  f_sum = 0
  for j in range(len(x)):
    yj_prev = 0 if j == 0 else y[j - 1]
    xj_prev = -1 if j == 0 else x[j - 1]
    f_sum += feature_function(w, theta, y[j], yj_prev, x[j], xj_prev)
  return f_sum

Compute normalizer Z by backtrack and dynamic programming

In [ ]:
def compute_log_Z_backtrack_step(w, theta, y, x, k):
  if k == -1:
    assert (len(y) == len(x)) 
    return 1
  else:
    sum_Z = 0
    for yj in range(1, n_states + 1):
      y = [yj] + y
      sum_Z += compute_log_Z_backtrack_step(w, theta, y, x, k - 1)
      y = y[1:]
  return sum_Z
    
      
def compute_log_Z_backtrack(w, theta, x):
  #return np.log(compute_log_Z_backtrack_step(w, theta, [], x, len(x) - 1))
  return compute_log_Z_backtrack_step(w, theta, [], x, len(x) - 1)

def compute_log_Z(w, theta, x):
  M = len(x)
  G = np.zeros((n_states, 1))
  sum_log_C = 0
  
  # case j = 0 (first word of x)
  for yj in range(1, n_states + 1):
    G[yj - 1, 0] = np.exp(feature_function(w, theta, yj, 0, x[0], -1))
  C = np.sum(G)
  G /= C
  sum_log_C += np.log(C)
    
  for j in range(1, M):
    G_j = np.zeros((n_states, n_states))
    for yj_prev in range(1, n_states + 1):
      for yj in range(1, n_states + 1):
        G_j[yj - 1, yj_prev - 1] = np.exp(feature_function(w, theta, yj, yj_prev, x[j], x[j - 1]))
    G = np.matmul(G_j, G)
    C = np.sum(G)
    G /= C
    sum_log_C += np.log(C)
  return np.log(np.sum(G)) + sum_log_C

In [14]:
i = 8
embedded_size = 2
w = np.random.rand(n_states * n_words + (n_states + 1) * n_states, 1)
theta = np.random.rand(n_words, embedded_size)
assert(compute_log_Z(w, theta, data[i]) - compute_log_Z_backtrack(w, theta, data[i]) < 1e-6)

In [125]:
def compute_log_conditional_prob(w, theta, y, x):
  return compute_F(w, theta, y, x) - compute_log_Z(w, theta, x)

def compute_minus_log_conditional_prob(w, theta, y, x):
  return -compute_log_conditional_prob(w, theta, y, x)

def compute_minus_log_conditional_prob_batch(w, theta, y_batches, x_batches):
  batch_prob = 0
  for y, x in zip(y_batches, x_batches):
    batch_prob += compute_minus_log_conditional_prob(w, theta, y, x)
  return batch_prob

def cost_regularized(w, theta, y_batches, x_batches, lambda_w, lambda_theta):
  return lambda_w * np.dot(w.T, w)[0,0] + lambda_theta * np.sum(theta * theta) + \
         compute_minus_log_conditional_prob_batch(w, theta, y_batches, x_batches)

### 2 Gradient

Numerical gradient

In [15]:
def numerical_grad(w, theta, cost, *args):
  grad_w = np.zeros_like(w)
  grad_theta = np.zeros_like(theta)
  eps = 1e-4
  for i in range(len(w)):
    w_high = w.copy()
    w_low = w.copy()
    w_high[i] += eps
    w_low[i] -= eps
    grad_w[i] = (cost(w_high, theta, *args) - cost(w_low, theta, *args)) / (2 * eps)
  for i in range(theta.shape[0]):
    for j in range(theta.shape[1]):
      theta_high = theta.copy()
      theta_low = theta.copy()
      theta_high[i, j] += eps
      theta_low[i, j] -= eps
      grad_theta[i, j] = (cost(w, theta_high, *args) - cost(w, theta_low, *args)) / (2 * eps)
  return grad_w, grad_theta

def check_grad(w, theta, cost, grad, *args):
  num_grad_w, num_grad_theta = numerical_grad(w, theta, cost,  *args)
  real_grad_w, real_grad_theta = grad(w, theta, *args)
  if np.linalg.norm(real_grad_w - num_grad_w) < 1e-6  and np.linalg.norm(real_grad_theta - num_grad_theta) < 1e-6:
    return True
  return False

Compute gradient wrt F

In [16]:
def compute_gradient_wrt_F(w, theta, y, x):
  grad_w = np.zeros((n_states * n_words + (n_states + 1) * n_states, 1))
  grad_theta = np.zeros((n_words, embedded_size))
  for j in range(len(x)):
    y_prev = 0 if j == 0 else y[j - 1]
    p1, p2 = transition_position(y_prev, y[j], x[j])
    grad_w[p1] += 1
    grad_w[p2] += 1
    if j > 0:
      sign = 1 if y[j] in in_word_state else -1
      grad_theta[x[j], :] += sign * theta[x[j - 1], :]
      grad_theta[x[j - 1], :] += sign * theta[x[j], :]
  return grad_w, grad_theta

In [17]:
embedded_size = 5
theta = np.random.rand(n_words, embedded_size)
y = sentence_states[6]
x = data[6]
assert(check_grad(w, theta, compute_F, compute_gradient_wrt_F, y, x))

NameError: name 'w' is not defined

Compute gradient wrt to the normalizer Z

In [ ]:
# Input: parameters w and encoded sentence x
# Output:
  # G runs from the transition 1 -> 2 to M-1 -> M (position not index in array)
  # alpha runs from j = 1 to j = M-1. For example, alpha[0][0] is the sum of prob of all length 1 sequence 
    # having the state = 0 (1 - 1)
  # beta runs from j = 2 to j = M. For example, beta[0][0] is the sum of prob of all length M-1 (from postion 
    # 2 to M) having the position 2 as state 0 (1 - 1)
def forward_backward(w, theta, x):
  alpha = np.zeros((n_states, 1))
  M = len(x)
  # case j = 0 (first word of x)
  for yj in range(1, n_states + 1):
    alpha[yj - 1, 0] = np.exp(feature_function(w, theta, yj, 0, x[0], -1))
  C = np.sum(alpha)
  alpha /= C
  alpha_list = [alpha]
  beta = np.ones((n_states, 1))
  beta_list = [beta]
  G_list = []
  
  # No need to calculate G and beta for length 1 string 
  if M == 1:
    alpha = np.concatenate(alpha_list, axis = 1)
    return alpha, [], []
  for j in range(1, M):
    G_j = np.zeros((n_states, n_states))
    for yj_prev in range(1, n_states + 1):
      for yj in range(1, n_states + 1):
        G_j[yj - 1, yj_prev - 1] = feature_function(w, theta, yj, yj_prev, x[j], x[j - 1])
    G_j = np.exp(G_j)
    G_list.append(G_j)
    if j < M - 1:
      alpha = np.matmul(G_j, alpha)
      C = np.sum(alpha)
      alpha /= C
      alpha_list.append(alpha)
  
  alpha = np.concatenate(alpha_list, axis = 1)
  G = np.stack(G_list)
  
  for j in range(M - 2, 0, -1):
    beta = np.matmul(G[j].T, beta)
    C = np.sum(beta)
    beta /= C
    beta_list = [beta] + beta_list
  beta = np.concatenate(beta_list, axis = 1)
  assert(len(alpha_list) == M - 1)
  assert(len(beta_list) == M - 1)
  assert(len(G_list) == M - 1)
  return alpha, beta, G

def compute_gradient_wrt_log_Z(w, theta, x):
  alpha, beta, G = forward_backward(w, theta, x)
  M = len(x)
  grad_w = np.zeros((n_states * n_words + (n_states + 1) * n_states, 1))
  grad_theta = np.zeros((n_words, embedded_size))
  # handle for the first position, need to find the sum of prob for each state in the first position
  if M == 1: # for length 1 string
    first_position_prob = alpha[:, 0]
  else:
    first_position_prob = np.multiply(alpha[:, 0], np.matmul(G[0].T, beta[:, 0]))
  first_position_prob /= np.sum(first_position_prob)
  for yj in range(1, n_states + 1):
      p1, p2 = transition_position(0, yj, x[0])
      grad_w[p1] += first_position_prob[yj - 1]
      grad_w[p2] += first_position_prob[yj - 1]
  if M == 1:
    return grad_w, grad_theta
  
  # handle for position 2 to M
  T = np.expand_dims(alpha.T, axis=1) * (G * np.expand_dims(beta.T, axis = 2))
  S = np.sum(np.sum(T, axis = 1), axis=1)
  S = np.expand_dims(np.expand_dims(S, axis = 1), axis = 1)
  T = T / S  
  
  for j in range(1, M): 
    for yj_prev in range(1, n_states + 1):
      for yj in range(1, n_states + 1):
        #prob_yj_prev_and_yj = alpha[:, j - 1][yj_prev - 1]*G[j-1][yj - 1, yj_prev - 1]*beta[:, j - 1][yj - 1]
        prob_yj_prev_and_yj = T[j - 1, yj - 1, yj_prev - 1]
        p1, p2 = transition_position(yj_prev, yj, x[j])
        grad_w[p1] += prob_yj_prev_and_yj
        grad_w[p2] += prob_yj_prev_and_yj
        sign = 1 if yj in in_word_state else -1
        grad_theta[x[j], :] += sign * prob_yj_prev_and_yj * theta[x[j - 1], :]
        grad_theta[x[j - 1], :] += sign * prob_yj_prev_and_yj * theta[x[j], :]
  return grad_w, grad_theta

In [20]:
i = 19
embedded_size = 2
w = np.random.rand(n_states * n_words + (n_states + 1) * n_states, 1)
theta = np.random.rand(n_words, embedded_size)
x = data[i]
num_grad_w, num_grad_theta = numerical_grad(w, theta, compute_log_Z, data[i])

In [44]:
import time
start = time.time()
grad_w, grad_theta = compute_gradient_wrt_log_Z(w, theta, x)
assert(np.linalg.norm(grad_w - num_grad_w) < 1e-6  and 
       np.linalg.norm(grad_theta - num_grad_theta) < 1e-6)
end = time.time()
print(end - start)

0.01889204978942871


Compute overall gradient

In [51]:
def compute_gradient_wrt_conditional_prob(w, theta, y, x):
  grad_w1, grad_theta1 = compute_gradient_wrt_F(w, theta, y, x)
  grad_w2, grad_theta2 = compute_gradient_wrt_log_Z(w, theta, x)
  return -grad_w1 + grad_w2, -grad_theta1 + grad_theta2

def compute_gradient_wrt_conditional_prob_batch(w, theta, y_batches, x_batches):
  grad_w = 0
  grad_theta = 0
  for y_batch, x_batch in zip(y_batches, x_batches):
    grad_w_batch, grad_theta_batch = compute_gradient_wrt_conditional_prob(w, theta, y_batch, x_batch)
    grad_w += grad_w_batch
    grad_theta += grad_theta_batch
  return grad_w, grad_theta

def grad_regularized(w, theta, y_batches, x_batches, lambda_w, lambda_theta):
  grad_w, grad_theta = compute_gradient_wrt_conditional_prob_batch(w, theta, y_batches, x_batches)
  return 2 * lambda_w * w + grad_w, 2 * lambda_theta * theta + grad_theta

### 3 Check gradient

In [24]:
i = 19
embedded_size = 2
w = np.random.rand(n_states * n_words + (n_states + 1) * n_states, 1)
theta = np.random.rand(n_words, embedded_size)
y_batches = sentence_states[i:i+1]
x_batches = data[i:i+1]
num_grad_w, num_grad_theta = numerical_grad(w, theta, cost_regularized, y_batches, x_batches, 0.5, 0.5)

In [26]:
import time
grad_w, grad_theta = grad_regularized(w, theta, y_batches, x_batches, 0.5, 0.5)
assert(np.linalg.norm(grad_w - num_grad_w) < 1e-6)
assert(np.linalg.norm(grad_theta - num_grad_theta) < 1e-6)

In [27]:
start = time.time()
grad_regularized(w, theta, sentence_states[:100], data[:100], 0.5, 0.5)
end = time.time()
print(end - start)

0.6771039962768555


### 4 Training

In [245]:
np.random.seed(500)
def generate_batch(X_train, y_train, n_batches):
  random_indices = np.random.permutation(len(X_train))
  X_train = X_train[random_indices]
  y_train = y_train[random_indices]
  X_batches = np.array_split(X_train, n_batches)
  y_batches = np.array_split(y_train, n_batches)
  return X_batches, y_batches 

eta = 0.001
n_epochs = 31
batch_size = 128
n_batches = len(X_train) // batch_size
beta1 = 0.9 
beta2 = 0.999
epsilon = 1e-8
lambda_w = 0.5
lambda_theta = 0.5

def crfGradientDescent(w_init, theta_init, cost, grad, adam = False):
  w = w_init
  theta = theta_init
  v_dw = 0
  s_dw = 0
  v_dtheta = 0
  s_dtheta = 0
  iteration = 1
  count_since_best = 0
  best_val_loss = 1e10
  best_w = w
  best_theta = theta
  for epoch in range(n_epochs):
    if epoch % 2 == 0:
      print("Epoch: ", epoch, " Val cost: ", cost(w, theta, y_val, X_val, 0, 0))
    X_batches, y_batches = generate_batch(X_train, y_train, n_batches)
    for X_batch, y_batch in zip(X_batches, y_batches):
      new_grad_w, new_grad_theta = grad(w, theta, y_batch, X_batch, lambda_w, lambda_theta)
      if adam:
        v_dw = beta1*v_dw + (1 - beta1)*new_grad_w
        s_dw = beta2*s_dw + (1 - beta2)*new_grad_w*new_grad_w
        v_dw_corrected = v_dw / (1 - beta1 ** iteration)
        s_dw_corrected = s_dw / (1 - beta2 ** iteration)
        w = w - eta * v_dw_corrected / (np.sqrt(s_dw_corrected) + epsilon)
        
        v_dtheta = beta1*v_dtheta + (1 - beta1)*new_grad_theta
        s_dtheta = beta2*s_dtheta + (1 - beta2)*new_grad_theta*new_grad_theta
        v_dtheta_corrected = v_dtheta / (1 - beta1 ** iteration)
        s_dtheta_corrected = s_dtheta / (1 - beta2 ** iteration)
        theta = theta - eta * v_dtheta_corrected / (np.sqrt(s_dtheta_corrected) + epsilon)
      else:
        w = w - eta*new_grad_w
        theta = theta - eta*new_grad_theta
      count_since_best += 1
      iteration += 1
      if iteration % 50 == 0:
        val_loss = cost(w, theta, y_val, X_val, 0, 0)
        if val_loss < best_val_loss:
          best_val_lost = val_loss
          count_since_best = 0
          best_w = w
          best_theta = theta
      if count_since_best > 800:
        print("Early stopping")
        return best_w, best_theta
  return w, theta

In [247]:
embedded_size = 15
# w_init = np.random.rand(n_states * n_words + (n_states + 1) * n_states, 1)
# theta_init = np.random.rand(n_words, embedded_size)
w, theta = crfGradientDescent(w, theta, cost_regularized, grad_regularized, adam = True)

Epoch:  0  Val cost:  2521.28464081
Epoch:  2  Val cost:  2540.27870753
Epoch:  4  Val cost:  2535.18831715
Epoch:  6  Val cost:  2528.06986244
Epoch:  8  Val cost:  2520.44799294
Epoch:  10  Val cost:  2514.13554136
Epoch:  12  Val cost:  2508.6990382
Epoch:  14  Val cost:  2502.66826391
Epoch:  16  Val cost:  2497.35829254
Epoch:  18  Val cost:  2491.20650822
Epoch:  20  Val cost:  2487.8266136
Epoch:  22  Val cost:  2482.69476205
Epoch:  24  Val cost:  2479.92171989
Epoch:  26  Val cost:  2477.4600676
Epoch:  28  Val cost:  2473.62604565
Epoch:  30  Val cost:  2469.69138629


In [249]:
np.savetxt('weight/crf_w.txt', w)
np.savetxt('weight/crf_theta.txt', theta)

In [248]:
cost_regularized(w, theta, y_val, X_val, 0, 0)

2468.7346481324739

In [153]:
w = np.loadtxt('weight/crf_w.txt', dtype=np.float64)
theta = np.loadtxt('weight/crf_theta.txt', dtype=np.float64)
w = w.reshape(-1, 1)

In [154]:
w.shape

(10512, 1)

### 5 Inference

In [161]:
# Vertibi algorithm for finding the best sequence
# Input:
  # w: parameters
  # x: encoded sentence
# Output:
  # sequence of best states
def find_best_sequence_crf(w, theta, x):
  M = len(x)
  G = np.zeros((n_states, 1))
  G_max_index = []
  # case j = 0 (first word of x)
  for yj in range(1, n_states + 1):
    G[yj - 1, 0] = feature_function(w, theta, yj, 0, x[0], -1)
  
  for j in range(1, M):
    G_j = np.zeros((n_states, n_states))
    for yj_prev in range(1, n_states + 1):
      for yj in range(1, n_states + 1):
        G_j[yj - 1, yj_prev - 1] = feature_function(w, theta, yj, yj_prev, x[j], x[j - 1])
    G = G_j + G.T
    G_max_index.append(np.expand_dims(np.argmax(G, axis=1), axis=1))
    G = np.max(G, axis = 1, keepdims = True)
  if len(G_max_index) > 0:
    G_max_index = np.concatenate(G_max_index, axis = 1)
    G_max_index_length = G_max_index.shape[1]
  else:
    G_max_index_length = 0
  current_max = np.argmax(G)
  best_sequence = [current_max + 1]
  for i in range(G_max_index_length - 1, -1, -1):
    current_max = G_max_index[current_max, i]
    best_sequence = [current_max + 1] + best_sequence
  return best_sequence

def find_best_sequence_batches(x_val, method = "", *args):
  if method == "crf":
    return [find_best_sequence_crf(*args, x) for x in x_val]
  elif method == "markov":
    return [find_best_sequence_markov(*args, x) for x in x_val]
  else:
    return [np.ones(len(x)) for x in x_val]

In [77]:
i = 353
#print(data[i])
#print(sentence_states[i])
#print(find_best_sequence_markov(data[i], A, O))
print(print_word_in_sentence(X_val[i], y_val[i], reverse_dictionary))
print('\n')
print(print_word_in_sentence(X_val[i], find_best_sequence_crf(w, theta, X_val[i]), reverse_dictionary))
print('\n')
print(print_word_in_sentence(X_val[i], find_best_sequence_markov(X_val[i], A, O), reverse_dictionary))

vđv bị truất quyền không được bất_kì giải_thưởng , huy_chương , cúp , xếp_hạng của toàn_bộ giải đấu


vđv bị truất quyền không được bất_kì giải_thưởng , huy_chương , cúp , xếp_hạng của toàn_bộ giải_đấu


vđv bị truất quyền không được bất_kì giải_thưởng , huy_chương , cúp , xếp_hạng của toàn_bộ giải_đấu


### 6 Evaluation

#### i) Scoring function

In [128]:
def score_sentence(y, y_pred):
  assert(len(y) == len(y_pred))
  index = 0
  TP = 0
  n_predicted_words = 0
  n_true_words = 0
  # find number of correct prediction and total number of predicted words
  while index < len(y_pred):
    old_index = index
    if y_pred[index] in start_word_state or y_pred[index] in only_word_state:  
      while index < len(y_pred) - 1 and y_pred[index + 1] in in_word_state:
        index += 1
    if index > old_index: # case find a multi-syllables word
      correct_prediction = y[old_index] in start_word_state
      for i in range(1, index - old_index + 1):
        if y[old_index + i] != y_pred[old_index + i]:
          correct_prediction = False
      if index < len(y_pred) - 1 and  y[index + 1] in in_word_state: # the true word is longer
        correct_prediction = False
    else: # case found a one syllable word
      correct_prediction = y[index] in only_word_state
    n_predicted_words += 1
    TP += correct_prediction
    index += 1
    
  # find total number of true words
  index = 0
  while index < len(y):
    if y[index] in start_word_state:  
      while index < len(y) - 1 and y[index + 1] in in_word_state:
        index += 1
    index += 1
    n_true_words += 1
  return TP, n_predicted_words, n_true_words
#   precision = TP / n_predicted_words
#   recall = TP / n_true_words
#   eps = 1e-8
#   F1 = 2 / (1/(precision + eps) + 1/(recall + eps))
#   return precision, recall, F1

def score_val(y_val, y_val_pred):
  TP, n_predicted_words, n_true_words = 0, 0, 0
  for index in range(len(y_val)):
    current_TP, current_n_predicted_words, current_n_true_words = score_sentence(y_val[index], y_val_pred[index])
    TP += current_TP
    n_predicted_words += current_n_predicted_words
    n_true_words += current_n_true_words
  precision = TP / n_predicted_words
  recall = TP / n_true_words
  eps = 1e-8
  F1 = 2 / (1/(precision + eps) + 1/(recall + eps))
  return precision, recall, F1

# def score_val2(y_val, y_val_pred):
#   precision, recall, F1 = 0, 0, 0
#   for index in range(len(y_val)):
#     current_precision, current_recall, current_F1 = score_sentence(y_val[index], y_val_pred[index])
#     precision += current_precision
#     recall += current_recall
#     F1 += current_F1
#   precision /= len(y_val)
#   recall /= len(y_val)
#   F1 /= len(y_val)
#   print(precision)
#   print(recall)
#   return precision, recall, F1

In [129]:
y =      [1, 1, 1, 2, 3, 3, 3, 3, 2, 3]
y_pred = [2, 3, 3, 2, 3, 3, 3, 3, 2, 3]
TP, n_predicted_words, n_true_words = score_sentence(y, y_pred)
print("TP: ", TP)
print("Predicted words: ", n_predicted_words)
print("True words: ", n_true_words)
assert(TP == 2)
assert(n_predicted_words == 3)
assert(n_true_words == 5)

TP:  2
Predicted words:  3
True words:  5


In [125]:
i = 1
print(print_word_in_sentence(X_val[i], y_val[i], reverse_dictionary))
print('\n')
print(print_word_in_sentence(X_val[i], find_best_sequence_crf(w, theta, X_val[i]), reverse_dictionary))

khu_vực doanh_nghiệp ( tư_nhân ) yếu


khu_vực doanh_nghiệp ( tư_nhân ) yếu


#### ii) Conditional random field evaluation

In [131]:
start = time.time()
y = y_val
y_pred = find_best_sequence_batches(X_val, "crf", w, theta)
precision, recall, F1 = score_val(y, y_pred)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", F1)
end = time.time()
print(end - start)

Precision:  0.9354320047389354
Recall:  0.9487597631104626
F1:  0.9420487572307712
1.0305149555206299


#### iii) Markov hidden model evaluation

In [68]:
y = y_val
y_pred = find_best_sequence_batches(X_val, "markov", A, O)
precision, recall, F1 = score_val(y, y_pred)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", F1)

Precision:  0.9151775077436264
Recall:  0.9301743622860833
F1:  0.922615006597802


#### iv) Baseline evaluation

In [69]:
y = y_val
y_pred = find_best_sequence_batches(X_val, "")
precision, recall, F1 = score_val(y, y_pred)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", F1)

Precision:  0.48573969759857694
Recall:  0.6612851146270584
F1:  0.5600793183393767


In [163]:
a = np.array([2, 3, 6, 4])
(-a).argsort()

array([2, 3, 1, 0])